## DOCKING

This jupyter notebook contains some example scripts to setup docking work for the adenosine A2B receptor.

First we prep the environment

In [4]:
!pip install oddt
!pip install vina
!jupyter labextension install jupyterlab_3dmol

/opt/conda/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Building jupyterlab assets (production, minimized)


In [5]:
#from pymol import cmd
import py3Dmol

import pandas as pd
import glob
import sys

from vina import Vina

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB


import nglview
from scripts import viewer
from scripts import autodock

import sys, os, shutil

import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False

There will be 4 stages in this notebook:
- 1.     protein preperation
- 2.     ligand preperation
- 3.     docking of a reference compound
- 4.     docking of your own compounds

# Stage 1

We have downloaded the inactive state model of the adenosine A2B receptor from gpcrdb:
https://gpcrdb.org/structure/homology_models/aa2br_human_inactive

We furthermore aligned this model to the adenosine A2A receptor, so we can use the same grid specification that was used for the docking introduction (lab 03 from CBR teaching).

In [6]:
# create a working directory
try:
    os.mkdir('workdir')
except:
    print('dir already there')

dir already there


In [7]:
# Here we prepare the AF model so we can use it for docking

command = 'bin/lepro data/A2b_AF_GPCRdb.pdb'
os.system(command)
shutil.move('pro.pdb','workdir/A2b_AF_GPCRdb-prepped.pdb')
os.remove('dock.in')

In [15]:
# Write it out to pdbqt
protein = Chem.MolFromPDBFile('workdir/A2b_AF_GPCRdb-prepped.pdb')

mol = Chem.Mol(protein)

#a = autodock.MolToPDBQTBlock(protein,flexible=False)
#with open('workdir/protein.pdbqt','w') as outfile:
#    for line in a:
#        outfile.write(line)

[13:39:58] Explicit valence for atom # 4 O, 4, is greater than permitted


ArgumentError: Python argument types in
    Mol.__init__(Mol, NoneType)
did not match C++ signature:
    __init__(_object*, RDKit::ROMol mol, bool quickCopy=False, int confId=-1)
    __init__(_object*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > pklString, unsigned int propertyFlags)
    __init__(_object*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > pklString)
    __init__(_object*)

In [17]:
# Next we will 
?scripts.autodock

# Stage 2

Generate 3D files of the molecules

In [ ]:
ligands = [
    
    'C1=COC(=C1)C2=NN3C(=NC(=NC3=N2)NCCC4=CC=C(C=C4)O)N', #ZMA241385
    'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'                        #Cafeine
]

In [ ]:
i = -1

IDs = []
for i in range(0,len(ligands)):
    base = 'ligand_{:02d}'.format(i)
    IDs.append(base)
    
df = pd.DataFrame({'ID':IDs, 'SMILES':test_smiles})

df['rdkit_mol'] = [Chem.MolFromSmiles(s) for s in df['SMILES']]

# Generate 2D images of the molecules
img=Draw.MolsToGridImage(df['rdkit_mol'].tolist(),
                            molsPerRow=4,
                            subImgSize=(200,200),
                            legends=df['ID'].tolist())

for i in range(len(df)):
    row = df.iloc[i]
    mol = row['rdkit_mol']
    name = row['ID'] + '.mol'
    Chem.MolToMolFile(mol,name)

img
